In [10]:
import glob
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
import tensorflow as tf
import re
import os

In [37]:
from tensorflow.keras.layers import MultiHeadAttention, Embedding

In [2]:
f = np.load('fall_detection_dataset.npz') 

In [3]:
trials = f['trials']


In [6]:
masking_layer = tf.keras.layers.Masking()

In [9]:
X_train, y_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [8]:
masked_embedding = masking_layer(trials)

Metal device set to: Apple M1 Pro


2022-11-25 20:27:26.622594: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-25 20:27:26.623413: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
mask = masking_layer.compute_mask(trials)

In [15]:
mask.shape

TensorShape([902, 500])

In [20]:
expanded_trials = tf.cast(
    tf.tile(tf.expand_dims(trials, axis=-2), [1, 1, 500,1]), tf.float32
)


In [ ]:
tf.keras.layers.MultiHeadAttention()

In [21]:
expanded_trials.shape

TensorShape([902, 500, 500, 3])

In [22]:
mask = masking_layer.compute_mask(expanded_trials)

In [23]:
mask.shape

TensorShape([902, 500, 500])

In [ ]:
tf.keras.Model.evaluate()

In [ ]:
tf.keras.Model.evaluate()

In [ ]:
tf.keras.Model.build(Q)

In [ ]:
tf.ad